In [32]:
from friendsfamilytest.params import *
from friendsfamilytest.sheethelper import *
from friendsfamilytest.data import *
from friendsfamilytest.utils import *

import pandas as pd 
import seaborn as sns

from wordcloud import WordCloud
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_formats='retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data = load_google_sheet()

In [ ]:
data.tail()

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
for index, row in data.iterrows():
    print(row['free_text'])
    print(index)

In [ ]:
def check_column_length(dataframe):
    short_phrases = []

    # Iterate over each entry in the 'do_better' column
    for entry in dataframe['free_text']:
        # Count the number of words in the entry
        word_count = len(str(entry).split())

        # Check if the word count is less than 5
        if word_count < 30:
            short_phrases.append(entry)

    # Print the list of entries with fewer than 5 words
    print("Entries with fewer than 5 words:", short_phrases)

# Example usage
# Assuming 'df' is your DataFrame and it has a column named 'do_better'
# df = pd.DataFrame({'do_better': ['Improve communication', 'Be more timely', 'Feedback', 'Enhance portal usability', 'Listen more']})
# check_do_better_column(df)

In [ ]:
check_column_length(data)

In [ ]:
def check_column_length(dataframe, column_name, word_count_length):
    # Iterate over each entry in the specified column
    for index, entry in enumerate(dataframe[column_name]):
        # Count the number of words in the entry
        word_count = len(str(entry).split())

        # Check if the word count is less than the specified limit
        if word_count < word_count_length:
            # Replace with NaN if the condition is met
            dataframe.at[index, column_name] = pd.nan

    return dataframe

In [ ]:
check_column_length(data2, 'do_better', 4)

In [ ]:
check_column_length(data2, 'free_text', 2)

In [22]:
data2 = pd.read_csv('../friendsfamilytest/data/data2.csv')

In [23]:
data2.head(30)

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound,feedback_labels
0,13/07/2023 15:28,Extremely likely,"Very friendly caring staff, treated me with re...",Keep it up,Earls Court Surgery,13,3,5.0,NaN,NaN,positive,0.909031,Follow-Up and Continuity of Care,0.487500,0.650000,0.124,0.413,0.463,0.8111,Overall Patient Satisfaction
1,13/07/2023 15:29,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4,0,5.0,NaN,NaN,positive,0.925523,NaN,0.700000,0.600000,0.000,0.484,0.516,0.4927,Patient Privacy and Confidentiality
2,13/07/2023 15:29,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8,5,5.0,NaN,NaN,positive,0.971687,Overall Patient Satisfaction,0.200000,0.300000,0.000,1.000,0.000,0.0000,Medical Staff Competence
3,13/07/2023 15:30,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5,5,5.0,NaN,NaN,positive,0.977053,Reception Staff Interaction,0.600000,1.000000,0.000,0.564,0.436,0.4754,Patient-Doctor Communication
4,13/07/2023 15:31,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44,6,5.0,NaN,NaN,neutral,0.761393,Overall Patient Satisfaction,0.118750,0.350000,0.134,0.866,0.000,-0.6757,Follow-Up and Continuity of Care
5,13/07/2023 15:32,Likely,NaN,NaN,Earls Court Surgery,0,0,4.0,NaN,NaN,neutral,1.000000,NaN,0.000000,0.000000,0.000,0.000,0.000,0.0000,NaN
6,13/07/2023 15:32,Extremely likely,Very professional and very kind. I feel comfor...,No. It is already perfect.,Earls Court Surgery,14,5,5.0,NaN,NaN,positive,0.975024,Overall Patient Satisfaction,0.436667,0.643333,0.000,0.579,0.421,0.8051,Overall Patient Satisfaction
7,13/07/2023 15:33,Extremely likely,Faster appointment & treatment ..,No,Earls Court Surgery,5,1,5.0,NaN,NaN,neutral,0.707314,Ambiance of Facility,0.000000,0.000000,0.000,1.000,0.000,0.0000,Referral Efficiency
8,13/07/2023 15:33,Extremely likely,Love the practice,NaN,Earls Court Surgery,3,0,5.0,NaN,NaN,positive,0.962944,NaN,0.500000,0.600000,0.000,0.323,0.677,0.6369,Overall Patient Satisfaction
9,13/07/2023 15:33,Extremely likely,The nurse Miss [PERSON] was absolutely amazing...,NaN,Earls Court Surgery,16,0,5.0,NaN,NaN,positive,0.984550,NaN,0.475000,0.660000,0.055,0.344,0.601,0.9504,Overall Patient Satisfaction


In [24]:
import numpy as np

In [25]:
data2.loc[data2['do_better_len'] < 5, 'do_better'] = np.nan
data2.loc[data2['do_better_len'] < 5, 'improvement_labels'] = np.nan

In [26]:
data2.head(10)

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound,feedback_labels
0,13/07/2023 15:28,Extremely likely,"Very friendly caring staff, treated me with re...",NaN,Earls Court Surgery,13,3,5.0,NaN,NaN,positive,0.909031,NaN,0.487500,0.650000,0.124,0.413,0.463,0.8111,Overall Patient Satisfaction
1,13/07/2023 15:29,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4,0,5.0,NaN,NaN,positive,0.925523,NaN,0.700000,0.600000,0.000,0.484,0.516,0.4927,Patient Privacy and Confidentiality
2,13/07/2023 15:29,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8,5,5.0,NaN,NaN,positive,0.971687,Overall Patient Satisfaction,0.200000,0.300000,0.000,1.000,0.000,0.0000,Medical Staff Competence
3,13/07/2023 15:30,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5,5,5.0,NaN,NaN,positive,0.977053,Reception Staff Interaction,0.600000,1.000000,0.000,0.564,0.436,0.4754,Patient-Doctor Communication
4,13/07/2023 15:31,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44,6,5.0,NaN,NaN,neutral,0.761393,Overall Patient Satisfaction,0.118750,0.350000,0.134,0.866,0.000,-0.6757,Follow-Up and Continuity of Care
5,13/07/2023 15:32,Likely,NaN,NaN,Earls Court Surgery,0,0,4.0,NaN,NaN,neutral,1.000000,NaN,0.000000,0.000000,0.000,0.000,0.000,0.0000,NaN
6,13/07/2023 15:32,Extremely likely,Very professional and very kind. I feel comfor...,No. It is already perfect.,Earls Court Surgery,14,5,5.0,NaN,NaN,positive,0.975024,Overall Patient Satisfaction,0.436667,0.643333,0.000,0.579,0.421,0.8051,Overall Patient Satisfaction
7,13/07/2023 15:33,Extremely likely,Faster appointment & treatment ..,NaN,Earls Court Surgery,5,1,5.0,NaN,NaN,neutral,0.707314,NaN,0.000000,0.000000,0.000,1.000,0.000,0.0000,Referral Efficiency
8,13/07/2023 15:33,Extremely likely,Love the practice,NaN,Earls Court Surgery,3,0,5.0,NaN,NaN,positive,0.962944,NaN,0.500000,0.600000,0.000,0.323,0.677,0.6369,Overall Patient Satisfaction
9,13/07/2023 15:33,Extremely likely,The nurse Miss [PERSON] was absolutely amazing...,NaN,Earls Court Surgery,16,0,5.0,NaN,NaN,positive,0.984550,NaN,0.475000,0.660000,0.055,0.344,0.601,0.9504,Overall Patient Satisfaction


In [27]:
data2.loc[data2['free_text_len'] < 3, 'free_text'] = np.nan
data2.loc[data2['free_text_len'] < 3, 'feedback_labels'] = np.nan

In [28]:
data2.head(30)

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound,feedback_labels
0,13/07/2023 15:28,Extremely likely,"Very friendly caring staff, treated me with re...",NaN,Earls Court Surgery,13,3,5.0,NaN,NaN,positive,0.909031,NaN,0.487500,0.650000,0.124,0.413,0.463,0.8111,Overall Patient Satisfaction
1,13/07/2023 15:29,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4,0,5.0,NaN,NaN,positive,0.925523,NaN,0.700000,0.600000,0.000,0.484,0.516,0.4927,Patient Privacy and Confidentiality
2,13/07/2023 15:29,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8,5,5.0,NaN,NaN,positive,0.971687,Overall Patient Satisfaction,0.200000,0.300000,0.000,1.000,0.000,0.0000,Medical Staff Competence
3,13/07/2023 15:30,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5,5,5.0,NaN,NaN,positive,0.977053,Reception Staff Interaction,0.600000,1.000000,0.000,0.564,0.436,0.4754,Patient-Doctor Communication
4,13/07/2023 15:31,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44,6,5.0,NaN,NaN,neutral,0.761393,Overall Patient Satisfaction,0.118750,0.350000,0.134,0.866,0.000,-0.6757,Follow-Up and Continuity of Care
5,13/07/2023 15:32,Likely,NaN,NaN,Earls Court Surgery,0,0,4.0,NaN,NaN,neutral,1.000000,NaN,0.000000,0.000000,0.000,0.000,0.000,0.0000,NaN
6,13/07/2023 15:32,Extremely likely,Very professional and very kind. I feel comfor...,No. It is already perfect.,Earls Court Surgery,14,5,5.0,NaN,NaN,positive,0.975024,Overall Patient Satisfaction,0.436667,0.643333,0.000,0.579,0.421,0.8051,Overall Patient Satisfaction
7,13/07/2023 15:33,Extremely likely,Faster appointment & treatment ..,NaN,Earls Court Surgery,5,1,5.0,NaN,NaN,neutral,0.707314,NaN,0.000000,0.000000,0.000,1.000,0.000,0.0000,Referral Efficiency
8,13/07/2023 15:33,Extremely likely,Love the practice,NaN,Earls Court Surgery,3,0,5.0,NaN,NaN,positive,0.962944,NaN,0.500000,0.600000,0.000,0.323,0.677,0.6369,Overall Patient Satisfaction
9,13/07/2023 15:33,Extremely likely,The nurse Miss [PERSON] was absolutely amazing...,NaN,Earls Court Surgery,16,0,5.0,NaN,NaN,positive,0.984550,NaN,0.475000,0.660000,0.055,0.344,0.601,0.9504,Overall Patient Satisfaction


In [ ]:
data2.shape

In [ ]:
# Saving the indexes corresponding to rows
boolean_mask = (data2['free_text_len']> 10)

# Applying the boolean filteraing
data3 = data2[boolean_mask].reset_index(drop=True)

In [ ]:
data3

In [ ]:
data3.columns

In [ ]:
data3[['time', 'rating', 'free_text', 'do_better', 'surgery', 'free_text_len',
       'do_better_len', 'rating_score', 'classif', 'classif_scores',
       'sentiment', 'sentiment_score', 'improvement_labels', 'polarity',
       'subjectivity', 'feedback_labels']]

In [ ]:
data3.drop(columns=['neg', 'neu', 'pos', 'compound'], inplace=True)

In [ ]:
data3

In [ ]:
import sketch
data3.sketch.ask("WHat is the average Neutral sentiment score?")

In [33]:
data = pd.read_csv('../friendsfamilytest/data/data.csv')

In [34]:
data

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound,feedback_labels
0,2023-07-13 15:28:22,Extremely likely,"Very friendly caring staff, treated me with re...",Keep it up,Earls Court Surgery,13,3,5.0,NaN,NaN,positive,0.909031,Follow-Up and Continuity of Care,0.48750,0.650,0.124,0.413,0.463,0.8111,Overall Patient Satisfaction
1,2023-07-13 15:29:16,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4,0,5.0,NaN,NaN,positive,0.925523,NaN,0.70000,0.600,0.000,0.484,0.516,0.4927,Patient Privacy and Confidentiality
2,2023-07-13 15:29:21,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8,5,5.0,NaN,NaN,neutral,0.971687,Overall Patient Satisfaction,0.20000,0.300,0.000,1.000,0.000,0.0000,Medical Staff Competence
3,2023-07-13 15:30:10,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5,5,5.0,NaN,NaN,positive,0.977053,Reception Staff Interaction,0.60000,1.000,0.000,0.564,0.436,0.4754,Patient-Doctor Communication
4,2023-07-13 15:31:48,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44,6,5.0,NaN,NaN,negative,0.761393,Overall Patient Satisfaction,0.11875,0.350,0.134,0.866,0.000,-0.6757,Follow-Up and Continuity of Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3607,2024-03-15 11:37:33,Unlikely,NaN,NaN,Earls Court Surgery,0,0,2.0,NaN,NaN,neutral,0.401299,NaN,0.00000,0.000,0.000,0.000,0.000,0.0000,NaN
3608,2024-03-15 12:23:16,Extremely likely,NaN,NaN,The Abingdon Health Centre,0,0,5.0,NaN,NaN,neutral,0.401299,NaN,0.00000,0.000,0.000,0.000,0.000,0.0000,NaN
3609,2024-03-17 16:01:28,Extremely likely,The surgery is very friendly and patient centr...,NaN,Earls Court Surgery,12,0,5.0,NaN,NaN,positive,0.973137,NaN,0.64375,0.825,0.000,0.582,0.418,0.8016,Overall Patient Satisfaction
3610,2024-03-17 16:32:48,Extremely likely,NaN,NaN,Earls Court Surgery,0,0,5.0,NaN,NaN,neutral,0.401299,NaN,0.00000,0.000,0.000,0.000,0.000,0.0000,NaN


In [35]:
data = clean_data(data)

2024-03-18 06:25:07.948 | INFO     | friendsfamilytest.utils:wrapper:100 - Function 'clean_data' ⚡️0.002669 sec


In [36]:
data.head(30)

,time,rating,free_text,do_better,surgery,free_text_len,do_better_len,rating_score,classif,classif_scores,sentiment,sentiment_score,improvement_labels,polarity,subjectivity,neg,neu,pos,compound,feedback_labels
0,2023-07-13 15:28:22,Extremely likely,"Very friendly caring staff, treated me with re...",NaN,Earls Court Surgery,13,3,5.0,NaN,NaN,positive,0.909031,NaN,0.487500,0.650000,0.124,0.413,0.463,0.8111,Overall Patient Satisfaction
1,2023-07-13 15:29:16,Extremely likely,so good so confidential,NaN,Earls Court Surgery,4,0,5.0,NaN,NaN,positive,0.925523,NaN,0.700000,0.600000,0.000,0.484,0.516,0.4927,Patient Privacy and Confidentiality
2,2023-07-13 15:29:21,Extremely likely,The doctor was very understanding and very kno...,No. It was the best.,Earls Court Surgery,8,5,5.0,NaN,NaN,neutral,0.971687,Overall Patient Satisfaction,0.200000,0.300000,0.000,1.000,0.000,0.0000,Medical Staff Competence
3,2023-07-13 15:30:10,Extremely likely,Dr [PERSON] was extremely nice,the receptionist was very unkind,Earls Court Surgery,5,5,5.0,NaN,NaN,positive,0.977053,Reception Staff Interaction,0.600000,1.000000,0.000,0.564,0.436,0.4754,Patient-Doctor Communication
4,2023-07-13 15:31:48,Extremely likely,I was spoken to politely on the phone and a te...,No. I was happy with everything.,Earls Court Surgery,44,6,5.0,NaN,NaN,negative,0.761393,Overall Patient Satisfaction,0.118750,0.350000,0.134,0.866,0.000,-0.6757,Follow-Up and Continuity of Care
5,2023-07-13 15:32:45,Likely,NaN,NaN,Earls Court Surgery,0,0,4.0,NaN,NaN,neutral,0.401299,NaN,0.000000,0.000000,0.000,0.000,0.000,0.0000,NaN
6,2023-07-13 15:32:52,Extremely likely,Very professional and very kind. I feel comfor...,No. It is already perfect.,Earls Court Surgery,14,5,5.0,NaN,NaN,positive,0.975024,Overall Patient Satisfaction,0.436667,0.643333,0.000,0.579,0.421,0.8051,Overall Patient Satisfaction
7,2023-07-13 15:33:01,Extremely likely,Faster appointment & treatment ..,NaN,Earls Court Surgery,5,1,5.0,NaN,NaN,neutral,0.707314,NaN,0.000000,0.000000,0.000,1.000,0.000,0.0000,Referral Efficiency
8,2023-07-13 15:33:02,Extremely likely,Love the practice,NaN,Earls Court Surgery,3,0,5.0,NaN,NaN,positive,0.962944,NaN,0.500000,0.600000,0.000,0.323,0.677,0.6369,Overall Patient Satisfaction
9,2023-07-13 15:33:33,Extremely likely,The nurse Miss [PERSON] was absolutely amazing...,NaN,Earls Court Surgery,16,0,5.0,NaN,NaN,positive,0.984550,NaN,0.475000,0.660000,0.055,0.344,0.601,0.9504,Overall Patient Satisfaction


In [37]:
data.to_csv('data.csv', index=False)